In [31]:
%reset -f

import numpy as np
import numba as  nb
import pandas as pd

all = pd.read_parquet('tmp.parq')
subset = all[-10000:]

In [44]:
@nb.jit(nopython=True)
def do_a_manual(t:np.array, val:np.array, bin_start:np.array, ) -> np.array:
    n_ticks, n_bins = len(t), len(np.unique(bin_start))
    i, j, ii = 0, 0, 0
    bin_index = np.full(n_bins, 0)
    n_samples = np.full(n_bins, 0)
    o = np.full(n_bins, 0.0)
    h = np.full(n_bins, 0.0)

    while j < n_ticks:
        while (j < n_ticks) & (bin_start[i] == bin_start[j]):
            j += 1
        # i is the index of the first element on the binned timestamp
        # je is the index to the last element of the binned timestamp
        # ii is the insertion point for the current bin

        je = j - 1
        bin_index[ii] = bin_start[je]
        n_samples[ii] = j - i
        o[ii] = val[i] #np.max()
        h[ii] = val[i] #np.max()
        ii += 1
        i = j
            
    return bin_index, n_samples, o, h

#pad_on_bin_bopundary
def foo(ticks:pd.DataFrame,
        bin_size_sec:int,
        time_col:str = "t",
        ) -> pd.DataFrame:

    tt = ticks[time_col].values.astype('datetime64[ms]').astype(np.int64)
    bin_start = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[ms]').astype(np.int64)
    val = (ticks.bid.values + ticks.ask.values) / 2
    bin_index, n_samples, o, h = do_a_manual(ticks.t.values, val, bin_start)
    return pd.DataFrame({'bin_start':bin_index.astype('datetime64[ms]'), 'n_samples':n_samples, 'o':o})

res = foo(subset, 60)
res[:5]

,bin_start,n_samples,o
0,2023-06-21 23:57:00,10,1.161865
1,2023-06-21 23:58:00,14,1.161855
2,2023-06-21 23:59:00,14,1.161875
3,2023-06-22 00:00:00,43,1.161815
4,2023-06-22 00:01:00,50,1.161885


In [47]:
res = foo(all, 60)
res

,bin_start,n_samples,o
0,2023-06-06 19:22:00,18,1.162185
1,2023-06-06 19:23:00,7,1.162195
2,2023-06-06 19:27:00,4,1.162165
3,2023-06-06 19:29:00,3,1.162165
4,2023-06-06 20:27:00,6,1.161995
...,...,...,...
9563,2023-06-22 06:02:00,38,1.160985
9564,2023-06-22 06:03:00,40,1.160965
9565,2023-06-22 06:04:00,30,1.161155
9566,2023-06-22 06:05:00,38,1.161275


In [ ]:
# def make_bin(x):
#     df = pd.Series()
#     t = x['t'].values
#     bid, ask, bsz, asz = x.bid.values, x.ask.values, x.bsz.values, x.asz.values
#     bin_end = x['bin_end'].values[0]
#     mid  = (bid + ask)/2.0
#     dt = np.diff(np.append(t, bin_end))


#     df['bin_end'] = bin_end 
#     df['o'] = mid[0]
#     df['h'] = np.max(mid)
#     df['l'] = np.min(mid)
#     df['c'] = mid[-1]
#     df['twap'] = np.sum(mid*dt)/np.sum(dt)
#     df['mean'] = np.mean(mid)

#     return df

# def gen_binned_data(ticks:pd.DataFrame,
#                     bin_size_sec:int,
#                     time_col:str = "t",
#                     ) -> pd.DataFrame:

#     # this is needed for twaps to work correctlly, otherwise we see downward bias in twaps
#     ticks[time_col] = ticks[time_col].values.astype('datetime64[ns]')

#     ticks['bin_start'] = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
#     ticks['bin_end'] = ticks['bin_start'].values + np.timedelta64(bin_size_sec, 's')

#     padded = ticks.groupby(['bin_start'], as_index=False).apply(make_bin).reset_index()
#     return padded

# subset = bid_ask[bid_ask.t > np.datetime64('2023-06-19 15:23:00')].copy()
# x = gen_binned_data(subset, 60)
# x